In [19]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from pathlib import Path

from core.image_io import get_image
from core.utils import Picture

In [45]:
input_dir = Path.cwd() / "data" / "raw"
output_dir = Path.cwd() / "data" / "output"
img = mpimg.imread(f"{input_dir}/{Picture.BOATS.value}")

In [46]:
type(img)

numpy.ndarray

In [47]:
red = img[:, :, 0]
green = img[:, :, 1]
blue = img[:, :, 2]
channels = {
    "red": red,
    "green": green,
    "blue": blue
}

In [48]:
for name, data in channels.items():
    plt.figure()
    plt.imshow(data, cmap="gray")
    plt.title(name)
    plt.axis('off')

    plt.savefig(f"{output_dir}/{name}_channel.png", dpi = 300)
    plt.close()